In [ ]:
fdep_path = "/Users/sakthi.n/Documents/Work/codegen/tmp"
from code_as_data.parsers.function_parser import FunctionParser
from code_as_data.parsers.class_parser import ClassParser
from code_as_data.parsers.import_parser import ImportParser
from code_as_data.parsers.type_parser import TypeParser
from code_as_data.parsers.instance_parser import InstanceParser


function_parser = FunctionParser(fdep_path)
class_parser = ClassParser(fdep_path)
import_parser = ImportParser(fdep_path)
type_parser = TypeParser(fdep_path)

functions_by_module = function_parser.load()
print("Starting class parsing...")
classes_by_module = class_parser.load()
print("Starting import parsing...")
imports_by_module = import_parser.load()
print("Starting type parsing...")
types_by_module = type_parser.load()
print("Processing instances...")
instance_parser = InstanceParser(fdep_path, fdep_path, functions_by_module)
instances_by_module = instance_parser.load_all_files()

In [ ]:
from code_as_data.networkx_adapter.networkx import NetworkxAdapter
graph = NetworkxAdapter(instances_by_module,types_by_module,imports_by_module,functions_by_module,classes_by_module)
graph.construct()
G = graph.get_graph_object()

In [4]:
def get_child_functions(G, node):
    funs = []
    for child in G.successors(node):
        if G.nodes[child].get("node_type","") == "Function" or G.nodes[child].get("type","") == "where_function":
            funs.append(child)
    return funs

In [ ]:
queue = [handler]
vis = set()

types = set()
while queue:
    node = queue.pop(0) 
    if node in vis:
        continue 
    vis.add(node)
    print(node)
    xs = G.nodes[node].get("function_signature","").split("->")
    types.update(xs)

    for child in get_child_functions(node):
        queue.append(child)

In [2]:
import networkx as nx
import pickle

with open('/Users/sakthi.n/Documents/Opensource/code-as-data/graph.pickle', 'rb') as file:
    g = pickle.load(file)


handler = "App.Routes:generateCheckList"
path = nx.shortest_path(g, handler, "OpenAIIntraction:requestGPT")

for path in path:
    print(path)

App.Routes:generateCheckList
GenIC:parsePDFDocAndStoreInGwDocTxt
OpenAIIntraction:requestGPT


In [5]:
queue = [handler]
vis = set()

vg = nx.DiGraph()

types = set()
while queue:
    node = queue.pop(0) 
    if node in vis:
        continue 
    vis.add(node)
    print(node)
    vg.add_node(node)
    # xs = g.nodes[node].get("function_signature","").split("->")
    # types.update(xs)

    for child in get_child_functions(g, node):

        if child not in vg.nodes():
            vg.add_node(child)
        queue.append(child)
        vg.add_edge(node, child)

App.Routes:generateCheckList
GenIC:parsePDFDocAndStoreInGwDocTxt
App.Routes:createIntegrationTracker
GenIC:validateIntegrationChecklist
OpenAIIntraction:requestGPT
App.Utils:extractTextFromPDF
Database.IntegrationChecklist.Types:toIntegrationChecklist
Database.IntegrationChecklist:updateDocDataIntegrationChecklist
Database.IntegrationChecklist:findOneIntegrationChecklist
Database.IntegrationChecklist.Types:createIntegrationChecklistDomain
Database.IntegrationChecklist:createIntegrationChecklist
GenIC:parsePDFDocAndStoreInGwDocTxt.url
GenIC:parsePDFDocAndStoreInGwDocTxt.promptMsg
GenIC:parsePDFDocAndStoreInGwDocTxt.gateway
GenIC:parsePDFDocAndStoreInGwDocTxt.icRep
GenIC:parsePDFDocAndStoreInGwDocTxt.baseDir
App.Utils:getCurrTime
Database.IntegrationTracker:createIntegrationTracker
App.Utils:getRandomUUID
App.Routes:createIntegrationTracker.integrationTracker
App.Routes:createIntegrationTracker.metadata
OpenAIIntraction:requestGPT.statusObject
OpenAIIntraction:requestGPT.timeoutSecs
Open

In [12]:
queue = [(handler, 0)]
vis = set()

while queue:
    node,lvl  = queue.pop(0) 
    
    if node in vis:
        continue
    vis.add(node)

    print(f"Level {lvl}: {node}")

    for child in get_child_functions(g, node):
        queue.append((child, lvl+1))
    

Level 0: App.Routes:generateCheckList
Level 1: GenIC:parsePDFDocAndStoreInGwDocTxt
Level 1: App.Routes:createIntegrationTracker
Level 2: GenIC:validateIntegrationChecklist
Level 2: OpenAIIntraction:requestGPT
Level 2: App.Utils:extractTextFromPDF
Level 2: Database.IntegrationChecklist.Types:toIntegrationChecklist
Level 2: Database.IntegrationChecklist:updateDocDataIntegrationChecklist
Level 2: Database.IntegrationChecklist:findOneIntegrationChecklist
Level 2: Database.IntegrationChecklist.Types:createIntegrationChecklistDomain
Level 2: Database.IntegrationChecklist:createIntegrationChecklist
Level 2: GenIC:parsePDFDocAndStoreInGwDocTxt.url
Level 2: GenIC:parsePDFDocAndStoreInGwDocTxt.promptMsg
Level 2: GenIC:parsePDFDocAndStoreInGwDocTxt.gateway
Level 2: GenIC:parsePDFDocAndStoreInGwDocTxt.icRep
Level 2: GenIC:parsePDFDocAndStoreInGwDocTxt.baseDir
Level 2: App.Utils:getCurrTime
Level 2: Database.IntegrationTracker:createIntegrationTracker
Level 2: App.Utils:getRandomUUID
Level 2: App.R

In [52]:
def all_child_expectone(node, child):
    childs = [] 
    metainfo = ""

    queue = [node]
    vis = set()  
    while queue: 
        cnode = queue.pop(0) 
        if cnode in vis:
            continue 
        vis.add(cnode)

        if g.nodes[cnode].get("behavior_summary", None):
            metainfo += f"FunctionName: {cnode}\nshortsummary: {g.nodes[cnode]["behavior_summary"]}\npreconditions: \n{"\n".join(g.nodes[node]["preconditions"])}\nposconditions: \n{"\n".join(g.nodes[node]["postconditions"])}\nsideeffects: \n{"\n".join(g.nodes[node]["side_effects"])}\n"
            state_deps = g.nodes[cnode].get("state_dependencies",None)
            if state_deps:
                metainfo += f"DB: {state_deps.get("db","None")}\nEnv: {state_deps.get("env","None")}\nRedis: {state_deps.get("redis","None")}\n"
            else:
                metainfo += "DB: None\nEnv: None\nRedis: None\n"

        for c in get_child_functions(g, cnode):
            if c != child:
                queue.append(c)
                childs.append(c)
        

    return childs, metainfo

def all_childs(node):
    childs = [] 
    metainfo = ""

    queue = [node]
    vis = set()  
    while queue: 
        cnode = queue.pop(0) 
        if cnode in vis:
            continue 
        vis.add(cnode)

        if g.nodes[cnode].get("behavior_summary", None):
            metainfo += f"FunctionName: {cnode}\nshortsummary: {g.nodes[cnode]["behavior_summary"]}\npreconditions: \n{"\n".join(g.nodes[node]["preconditions"])}\nposconditions: \n{"\n".join(g.nodes[node]["postconditions"])}\nsideeffects: \n{"\n".join(g.nodes[node]["side_effects"])}\n"
            state_deps = g.nodes[cnode].get("state_dependencies",None)
            if state_deps:
                metainfo += f"DB: {state_deps.get("db","None")}\nEnv: {state_deps.get("env","None")}\nRedis: {state_deps.get("redis","None")}\n"
            else:
                metainfo += "DB: None\nEnv: None\nRedis: None\n"

        for c in get_child_functions(g, cnode):
            queue.append(c)
            childs.append(c)
        

    return childs,metainfo

In [55]:
(_, s) = all_child_expectone("GenIC:parsePDFDocAndStoreInGwDocTxt","OpenAIIntraction:requestGPT")
# (_, s) = all_childs("OpenAIIntraction:requestGPT")
print(s)

FunctionName: GenIC:parsePDFDocAndStoreInGwDocTxt
shortsummary: Extracts text from a provided PDF, and uses it to either create a new integration checklist via an LLM or update an existing one in the database.
preconditions: 
Input `docData` must contain a PDF payload in the `gwDocPdf` field.
The application must have file system permissions to create directories and write/delete files.
External PDF text extraction utility `Utils.extractTextFromPDF` must be functional.
LLM API endpoint (GPT-4 Turbo) must be configured and network-accessible.
posconditions: 
If no checklist exists for the gateway, a new one is created in the `integration_checklists` table using data generated by an LLM.
If a checklist exists for the gateway, it is updated in the database with the text extracted from the PDF.
A temporary PDF file written to the file system is deleted.
Returns the created or updated integration checklist and a validation result.
sideeffects: 
Creates a temporary directory on the file syst

In [ ]:
from google import genai
from google.genai import types

def call_gemini(user_prompt: str, system_prompt=None, response_format="application/json", response_schema=None):
    client = genai.Client(
        api_key=""
    )

    model = "gemini-2.5-pro-preview-06-05"
    contents = [
        types.Content(
            role="user",
            parts=[types.Part.from_text(text=user_prompt)],
        ),
    ]

    generate_content_config = types.GenerateContentConfig(
        system_instruction=system_prompt,
        response_mime_type=response_format,
        response_schema=response_schema,
    )

    response = client.models.generate_content(
                model=model,
                config=generate_content_config,
                contents=contents,
            )
    return response.text

In [ ]:

handler = "App.Routes:generateCheckList"
path = nx.shortest_path(g, handler, "OpenAIIntraction:requestGPT")
prev_context = ""
path_str = "" 

for node in path:
    path_str += node + " -> "
path_str = path_str.removesuffix(" -> ")

for idx,node in enumerate(path): 
    if idx+1 == len(path):
        (_,metainfo) = all_childs(node)
    else: 
        (_,metainfo) = all_child_expectone(node, path[idx+1])
    
    prompt = user_prompt.format(prev_context=prev_context,current_function=node,metadata=metainfo,path=path_str)

    print(prompt)
    # result = call_gemini(user_prompt=prompt,response_format="text/plain")

    # print(result)
    break



You are helping generate a backend functional test case by walking through a call chain of Haskell functions.

This is a step-by-step exploration.

In the previous step, we explored other descendant functions (excluding this subgraph) and gathered the following test setup information:

---
# Previous setup context (accumulated preconditions/state):

---

Now we are analyzing the function: `App.Routes:generateCheckList`.

Here is its metadata:

FunctionName: App.Routes:generateCheckList
shortsummary: Parses PDF data, stores its text content in a gateway document, and creates an associated integration tracker record.
posconditions: 
A new record is created storing the parsed text of the PDF.
A new integration tracker record is created for this operation.
Returns an identifier for the newly created gateway document text record.
sideeffects: 
Inserts a record into the gateway document text table (inferred from `parsePDFDocAndStoreInGwDocTxt`).
Inserts a record into the integration tracker

In [ ]:
path_str = "" 

for node in path:
    path_str += node + " -> "
path_str = path_str.removesuffix(" -> ")

In [21]:
g.nodes["OpenAIIntraction:requestGPT"]

{'label': 'requestGPT',
 'node_type': 'Function',
 'function_signature': 'Maybe Double\n-> Maybe Int\n-> Maybe Double\n-> Maybe Double\n-> [Message]\n-> Text\n-> [(HeaderName, ByteString)]\n-> Maybe ResponseFormat\n-> Maybe [ImageBody]\n-> IO (Either (Int, Text) Text)',
 'code_string': 'requestGPT\n  temp\n  maxTokens\n  frequencyPenalty\n  presencePenalty\n  promptMsg\n  url\n  headers\n  responseFormat\n  imgListM\n  = do let imgList\n             = case imgListM of\n                 Nothing -> []\n                 Just lst -> map Left lst\n       manager <- newManager tlsManagerSettings\n       let timeoutSecs = oaiReqTimeoutSecs\n           requestBody\n             = LLMRequestBody\n                 {messages = (map Right promptMsg) <> imgList,\n                  max_tokens = Just $ fromMaybe 16384 maxTokens, temperature = temp,\n                  frequency_penalty = frequencyPenalty,\n                  presence_penalty = presencePenalty, top_p = 0, stop = Nothing,\n              

In [15]:
handler = "App.Routes:generateCheckList"
path = nx.shortest_path(g, handler, "OpenAIIntraction:requestGPT")

for node in path:
    print(path)



['App.Routes:generateCheckList', 'GenIC:parsePDFDocAndStoreInGwDocTxt', 'OpenAIIntraction:requestGPT']
['App.Routes:generateCheckList', 'GenIC:parsePDFDocAndStoreInGwDocTxt', 'OpenAIIntraction:requestGPT']
['App.Routes:generateCheckList', 'GenIC:parsePDFDocAndStoreInGwDocTxt', 'OpenAIIntraction:requestGPT']
